In [ ]:
import numpy
import pandas as pd
import os
import csv
import ast
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.utils import assets_directory
from openfisca_france_indirect_taxation.examples.utils_example import (
    wavg,
    collapse,
    dataframe_by_group,
    graph_builder_bar,
    df_weighted_average_grouped)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.almost_ideal_demand_system.utils import add_niveau_vie_decile
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [ ]:
from openfisca_france_indirect_taxation.Calage_consommation_bdf import (
    get_cn_aggregates,
    get_inflators_by_year)

In [ ]:
get_cn_aggregates(2018)

In [ ]:
input = get_input_data_frame(2017)

In [ ]:
inflators = get_inflators_by_year(rebuild = True, year_range=[2024],data_year = 2017)

In [ ]:
input['rev701_pond'] = input['somme_libre_recue']*input['pondmen']
input['rev701_pond'].sum()

In [ ]:
float(inflators.get(2024).get('rev_disponible') * input['rev701_pond'].sum())

In [ ]:
input.loc[input['revact'] != input['act_indpt'] + input['salaires'], ['revact','act_indpt','salaires']]

In [ ]:
input['rev_etranger'] = input['revact'] - input['act_indpt'] - input['salaires']
input['rev_etranger_pond'] = input['rev_etranger'] * input['pondmen']
float( inflators.get(2024).get('rev_disponible') * input['rev_etranger_pond'].sum())

In [ ]:
data_year = 2017
inflators_by_year = get_inflators_by_year(rebuild = True, year_range = range(2017, 2025), data_year = data_year)
simulated_variables = ['tva_taux_plein',
'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'depenses_tva_exonere',
 'depenses_tva_taux_plein',
 'depenses_tva_taux_intermediaire',
 'depenses_tva_taux_reduit',
 'depenses_tva_taux_super_reduit',
 'rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
]

agregates = pd.DataFrame(columns = simulated_variables)
for year in range(2017,2025) :
    inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])
    survey_scenario = SurveyScenario.create(
        inflation_kwargs = inflation_kwargs,
        year = year,
        data_year = data_year
        )
    df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
    agregates.loc[year] = df_sum.sum()

In [ ]:
agregates 

In [ ]:
agregates[['depenses_tva_taux_plein', 'depenses_tva_taux_intermediaire' , 'depenses_tva_taux_reduit','depenses_tva_taux_super_reduit','depenses_tva_exonere']]

In [ ]:
tva_cpo = pd.DataFrame(data = {'' : [2015,2016,2017,2018,2019,2020,2021] , 'tva (CPO)' : [152.6, 155.4, 162.9, 168.6, 174.9, 162.6, 185.9]})
tva_cpo.set_index('', inplace= True)

In [ ]:
tva_cpo.loc[:,'tva (CPO)'] = tva_cpo.loc[:,'tva (CPO)'] * 1e9

In [ ]:
compare_agregate = tva_cpo.merge(agregates['tva_total'], left_index = True, right_index = True)
compare_agregate.rename(columns={'tva_total' : 'tva (BdF)'}, inplace = True)
compare_agregate = (compare_agregate * 1e-9).round(1)
compare_agregate.loc[:,'ratio'] = (compare_agregate.loc[:,'tva (BdF)'] / compare_agregate.loc[:,'tva (CPO)'] * 100).round(2) 

In [ ]:
compare_agregate

In [ ]:
compare_agregate['growth rate (CPO)'] = compare_agregate['tva (CPO)'].pct_change()
compare_agregate['growth rate (BDF)'] = compare_agregate['tva (BdF)'].pct_change()
compare_agregate

In [ ]:
tva_plf= pd.DataFrame(data = {'' : [2021,2022,2023,2024] , 'tva (Compta Nat)' : [186.7,202.7,208.4,210.1]})
tva_plf.set_index('', inplace= True)
tva_plf.loc[:,'tva (Compta Nat)'] = tva_plf.loc[:,'tva (Compta Nat)'] * 1e9

In [ ]:
compare_agregate_plf = tva_plf.merge(agregates['tva_total'], left_index = True, right_index = True)
compare_agregate_plf.rename(columns={'tva_total' : 'tva (BdF)'}, inplace = True)
compare_agregate_plf = compare_agregate_plf * 1e-9
compare_agregate_plf = compare_agregate_plf.round(1)
compare_agregate_plf.loc[:,'ratio'] = (compare_agregate_plf.loc[:,'tva (BdF)'] / compare_agregate_plf.loc[:,'tva (Compta Nat)'] * 100).round(2) 

In [ ]:
compare_agregate_plf

In [ ]:
parametres_fiscalite_file_path = os.path.join(
            assets_directory,
            'legislation',
            'T_3217.xlsx'
            )

In [ ]:
compta_nat = pd.read_excel(parametres_fiscalite_file_path, header = 2)
compta_nat.drop('Unnamed: 0', axis = 1, inplace = True)
compta_nat = compta_nat.loc[compta_nat.index == 1].T.drop('Unnamed: 1', axis = 0)
compta_nat.rename({1 : 'tva (Compta Nat)'}, axis = 1, inplace = True )
compta_nat = compta_nat * 1E9

In [ ]:
compta_nat.loc[compta_nat.index == 2015]

In [ ]:
compare_agregate_compta_nat = compta_nat.merge(agregates['tva_total'], left_index = True, right_index = True)
compare_agregate_compta_nat.rename(columns={'tva_total' : 'tva (BdF)'}, inplace = True)
compare_agregate_compta_nat = compare_agregate_compta_nat * 1E-9
compare_agregate_compta_nat = compare_agregate_compta_nat.round(1)
compare_agregate_compta_nat.loc[:,'ratio'] = (compare_agregate_compta_nat.loc[:,'tva (BdF)'] / compare_agregate_compta_nat.loc[:,'tva (Compta Nat)'] * 100)

In [ ]:
compare_agregate_compta_nat

## Le revenu disponible et les niveaux de vie

In [ ]:
data_year = 2017 
survey_scenario_test = SurveyScenario.create(
    year = 2017,
    data_year = data_year
    )
test_revenus_2017= dataframe_by_group(survey_scenario_test, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'mean')

In [ ]:
survey_scenario.compute_pivot_table

In [ ]:
test_table_micro_2017 = survey_scenario_test.create_data_frame_by_entity(simulated_variables+['situapr','decile_rev_disponible','situacj'], period = 2017) 
test_menages_2017 = test_table_micro_2017['menage']

In [ ]:
mean_by_rev_decile = df_weighted_average_grouped(dataframe = test_menages_2017[test_menages_2017['situapr'] != 3], groupe = 'decile_rev_disponible', varlist = simulated_variables + ['situapr'])

In [ ]:
mean_by_rev_decile['rev_disponible']

In [ ]:
data_year = 2017
inflators_by_year = get_inflators_by_year(rebuild = False, year_range = range(2017, 2025), data_year = data_year)
simulated_variables = ['rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'niveau_de_vie',
 'ocde10',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
]

inflation_kwargs = dict(inflator_by_variable = inflators_by_year[2018])
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2018,
    data_year = data_year
    )

In [ ]:
revenus_mean_2018 = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'mean')

In [ ]:
revenus_mean_2018

In [ ]:
revenus_mean_2018_by_rev_decile = dataframe_by_group(survey_scenario, category = 'decile_rev_disponible', variables = simulated_variables, aggfunc = 'mean')

In [ ]:
revenus_mean_2018_by_rev_decile

In [ ]:
table_micro = survey_scenario.create_data_frame_by_entity(
    ['rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'niveau_de_vie',
 'niveau_vie_decile',
 'decile_rev_disponible',
 'ocde10',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
 'situapr'
], 
    period = 2018) 
menages = table_micro['menage']

Y a-t-il des ménages dont le responsable est étudiant ? (situapr == 3)

In [ ]:
len(menages[menages['situapr'] == 3])

Y a-t-il des ménages dont le revenu disponible est négatif ou nul ?

In [ ]:
len(menages[menages['rev_disponible'] <= 0])

### Revenu disponible par décile

In [ ]:
mean_by_rev_decile = df_weighted_average_grouped(dataframe =  menages[(menages['situapr'] != 3) & (menages['rev_disponible'] > 0)], groupe = 'decile_rev_disponible', varlist = simulated_variables)
mean_by_rev_decile

In [ ]:
revenus_mean_2018_by_rev_decile

### Niveaux de vie par décile

In [ ]:
df_weighted_average_grouped(dataframe= menages, groupe = 'niveau_vie_decile', varlist = simulated_variables)

In [ ]:
revenus_mean_2018

## Structure de la consommation par type de taux de TVA, par décile de niveau de vie

In [ ]:
simulated_variables = ['depenses_tva_taux_plein',
'depenses_tva_taux_intermediaire',
'depenses_tva_taux_reduit',
'depenses_tva_taux_super_reduit',
'depenses_tva_exonere',
'depenses_totales',
'loyer_impute',
'tva_taux_plein',
 'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'niveau_de_vie',
 'rev_disponible',
 'decile_rev_disponible',
 'rev_disp_yc_loyerimpute',
 'depenses_tot',
 'pondmen'] + ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_{}".format(i) for i in range(10, 14)] + ['poste_04_1_1_1',
 'poste_04_1_2_1',
 'poste_04_3_1_1',
 'poste_04_3_2_1',
 'poste_04_4_1_1',
 'poste_04_4_2_1',
 'poste_04_4_3_1',
 'poste_04_4_4_1',
 'poste_04_5_0_0',
 'poste_04_5_1_1',
 'poste_04_5_2_1',
 'poste_04_5_2_2',
 'poste_04_5_3_1',
 'poste_04_5_4_1',
 'poste_04_5_5_1',
 'poste_04_5_5_2',
 'poste_04_6_1_1']

In [ ]:
inflators_by_year = get_inflators_by_year(rebuild = False, year_range = range(2017, 2025), data_year = data_year)
inflation_kwargs = dict(inflator_by_variable = inflators_by_year[2017])
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2017,
    data_year = data_year
    )
to_graph = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'mean')
to_graph['poste_agrege_12'] += to_graph['poste_agrege_13']
to_graph.drop('poste_agrege_13', axis = 1, inplace = True)

table_micro_2017 = survey_scenario.create_data_frame_by_entity(simulated_variables, period = 2017) 
menages_2017 = table_micro_2017['menage']

### La non-réponse dans l'enquête Budget de famille (2017)

In [ ]:
len(menages_2017)

In [ ]:
liste_postes_agreges = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_{}".format(i) for i in range(10, 13)]

In [ ]:
{ key : dict((menages_2017== 0).sum())[key] for key in liste_postes_agreges}

In [ ]:
menages_2017.loc[menages_2017['depenses_tot'] > menages_2017['rev_disponible'], ['depenses_tot','rev_disponible'] + liste_postes_agreges ]

In [ ]:
test = menages_2017.loc[menages_2017['poste_04_3_2_1'] > 0, ['poste_04_3_2_1','rev_disponible','decile_rev_disponible']]

In [ ]:
test.sort_values(by = 'poste_04_3_2_1', ascending = False)

In [ ]:
test.loc[test['decile_rev_disponible'] == 6,'poste_04_3_2_1'].describe()

In [ ]:
test.loc[test['decile_rev_disponible'] == 7,'poste_04_3_2_1'].describe()

In [ ]:
test.loc[test['decile_rev_disponible'] == 9,'poste_04_3_2_1'].describe()

### Structure de la consommation par poste agrégé

In [ ]:
label_postes_agreges = ['Alimentation', 'Alcools et tabacs', 'Habillement et chaussures', 'Logement', 'Ameublement et équipement ménager', 'Santé',
                        'Transports', 'Communication' , 'Loisirs et cultures' , 'Education', 'Restauration et hôtellerie', 'Services et biens divers']
liste_postes_agreges = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_{}".format(i) for i in range(10, 13)]

In [ ]:
to_graph

In [ ]:
for poste in liste_postes_agreges:
    to_graph['Part poste {}'.format(poste[-2:])] = to_graph[poste] / to_graph['depenses_tot']

In [ ]:
to_graph.index = to_graph.index.astype(int)

In [ ]:
to_graph

In [ ]:
from openfisca_france_indirect_taxation.projects.TVA.Utils import stacked_bar_plot

In [ ]:
stacked_bar_plot(to_graph, 
                 variables = ["Part poste 0{}".format(i) for i in range(1, 10)] + ["Part poste {}".format(i) for i in range(10, 13)],
                 labels = label_postes_agreges,
                 title = 'Structure de la consommation par postes agrégés',
                 xlabel = 'Décile de niveau de vie' ,
                 ylabel = '',
                 colors = list(sns.color_palette("Paired")),
                 savefig = True,
                 outfile = 'Structure_conso_postes_agreges.pdf')

In [ ]:
to_graph[['poste_04_1_1_1',
 'poste_04_1_2_1',
 'loyer_impute',
 'poste_04_3_1_1',
 'poste_04_3_2_1',
 'poste_04_4_1_1',
 'poste_04_4_2_1',
 'poste_04_4_3_1',
 'poste_04_4_4_1',
 'poste_04_5_0_0',
 'poste_04_5_1_1',
 'poste_04_5_2_1',
 'poste_04_5_2_2',
 'poste_04_5_3_1',
 'poste_04_5_4_1',
 'poste_04_5_5_1',
 'poste_04_5_5_2',
 'poste_04_6_1_1', 
 'depenses_tot']]

In [ ]:
to_graph['loyers'] = to_graph['poste_04_1_1_1'] + to_graph['poste_04_1_2_1'] + to_graph['loyer_impute']
to_graph['part loyer'] = to_graph['loyers'] / to_graph['depenses_tot']
to_graph[['loyers', 'depenses_tot', 'part loyer']]

### Structure de la consommation par taux de TVA 

In [ ]:
to_graph[['depenses_tva_taux_plein' , 'depenses_tva_taux_intermediaire', 'depenses_tva_taux_reduit','depenses_tva_taux_super_reduit','depenses_tva_exonere','depenses_tot']]

In [ ]:
to_graph.loc[:,'Part TVA exonere'] = to_graph['depenses_tva_exonere'] / to_graph['depenses_tot']
to_graph.loc[:,'Part TVA taux super réduit'] = to_graph['depenses_tva_taux_super_reduit'] / to_graph['depenses_tot']
to_graph.loc[:,'Part TVA taux réduit'] = to_graph['depenses_tva_taux_reduit'] / to_graph['depenses_tot']
to_graph.loc[:,'Part TVA taux intermédiaire'] = to_graph['depenses_tva_taux_intermediaire'] / to_graph['depenses_tot']
to_graph.loc[:,'Part TVA taux plein'] = to_graph['depenses_tva_taux_plein'] / to_graph['depenses_tot']

In [ ]:
stacked_bar_plot(to_graph, 
                 variables = ['Part TVA taux plein','Part TVA taux intermédiaire','Part TVA taux réduit','Part TVA taux super réduit','Part TVA exonere'],
                 labels = ['Taux plein','Taux intermédiaire','Taux réduit','Taux super réduit','Taux exonere'],
                 title = 'Structure de la consommation par type de taux de TVA',
                 xlabel = 'Décile de niveau de vie' ,
                 ylabel ='',
                 colors = list(sns.color_palette("Paired")),
                 savefig = True,
                 outfile = 'Structure_conso_taux_tva_avec_exo.pdf')

In [ ]:
to_graph.loc[:,'Part TVA taux super réduit'] = to_graph['depenses_tva_taux_super_reduit'] / to_graph['depenses_totales']
to_graph.loc[:,'Part TVA taux réduit'] = to_graph['depenses_tva_taux_reduit'] / to_graph['depenses_totales']
to_graph.loc[:,'Part TVA taux intermédiaire'] = to_graph['depenses_tva_taux_intermediaire'] / to_graph['depenses_totales']
to_graph.loc[:,'Part TVA taux plein'] = to_graph['depenses_tva_taux_plein'] / to_graph['depenses_totales']

In [ ]:
stacked_bar_plot(to_graph, 
                 variables = ['Part TVA taux plein','Part TVA taux intermédiaire','Part TVA taux réduit','Part TVA taux super réduit'],
                 labels = ['Taux plein','Taux intermédiaire','Taux réduit','Taux super réduit'],
                 title = 'Structure de la consommation par type de taux de TVA',
                 xlabel = 'Décile de niveau de vie' ,
                 ylabel ='',
                 colors = list(sns.color_palette("Paired")),
                 savefig = True,
                 outfile = 'Structure_conso_taux_tva.pdf')

In [ ]:
total_graph = pd.DataFrame(to_graph[['depenses_tva_taux_plein' , 'depenses_tva_taux_intermediaire', 
          'depenses_tva_taux_reduit','depenses_tva_taux_super_reduit',
          'depenses_tva_exonere','depenses_tot', 'depenses_totales','tva_total']].mean()).T

In [ ]:
total_graph.loc[:,'Part TVA exonere'] = total_graph['depenses_tva_exonere'] / total_graph['depenses_tot']
total_graph.loc[:,'Part TVA taux super réduit'] = total_graph['depenses_tva_taux_super_reduit'] / total_graph['depenses_tot']
total_graph.loc[:,'Part TVA taux réduit'] = total_graph['depenses_tva_taux_reduit'] / total_graph['depenses_tot']
total_graph.loc[:,'Part TVA taux intermédiaire'] = total_graph['depenses_tva_taux_intermediaire'] / total_graph['depenses_tot']
total_graph.loc[:,'Part TVA taux plein'] = total_graph['depenses_tva_taux_plein'] / total_graph['depenses_tot']

In [ ]:
total_graph[['Part TVA exonere','Part TVA taux super réduit','Part TVA taux réduit','Part TVA taux intermédiaire','Part TVA taux plein']]

In [ ]:
total_graph.loc[:,'Part TVA taux super réduit'] = total_graph['depenses_tva_taux_super_reduit'] / total_graph['depenses_totales']
total_graph.loc[:,'Part TVA taux réduit'] = total_graph['depenses_tva_taux_reduit'] / total_graph['depenses_totales']
total_graph.loc[:,'Part TVA taux intermédiaire'] = total_graph['depenses_tva_taux_intermediaire'] / total_graph['depenses_totales']
total_graph.loc[:,'Part TVA taux plein'] = total_graph['depenses_tva_taux_plein'] / total_graph['depenses_totales']

In [ ]:
total_graph[['Part TVA taux super réduit','Part TVA taux réduit','Part TVA taux intermédiaire','Part TVA taux plein']]

# Taux Effectif moyen TVA (2019)

In [ ]:
simulated_variables = ['depenses_tva_taux_plein',
'depenses_tva_taux_intermediaire',
'depenses_tva_taux_reduit',
'depenses_tva_taux_super_reduit',
'depenses_tva_exonere',
'depenses_totales',
'loyer_impute',
'tva_taux_plein',
 'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'niveau_de_vie',
 'rev_disponible',
 'decile_rev_disponible',
 'rev_disp_yc_loyerimpute',
 'depenses_tot',
 'pondmen']

In [ ]:
inflators_by_year = get_inflators_by_year(rebuild = False, year_range = range(2017, 2025), data_year = data_year)
inflation_kwargs = dict(inflator_by_variable = inflators_by_year[2017])
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2019,
    data_year = data_year,
    )
to_graph = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')

In [ ]:
tva_total = survey_scenario.compute_aggregate('tva_total', use_baseline= False, period = 2019)
depenses_totales = survey_scenario.compute_aggregate('depenses_totales', use_baseline= False, period = 2019)

In [ ]:
tva_total / depenses_totales

# Réforme TVA

In [ ]:
from openfisca_france_indirect_taxation.projects.TVA.Reform_TVA_budget_2025 import augmente_tous_les_taux

In [ ]:
simulated_variables = ['depenses_tva_taux_plein',
'depenses_tva_taux_intermediaire',
'depenses_tva_taux_reduit',
'depenses_tva_taux_super_reduit',
'depenses_ht_tva_taux_plein',
'depenses_ht_tva_taux_intermediaire',
'depenses_ht_tva_taux_reduit',
'depenses_ht_tva_taux_super_reduit',
'depenses_tva_exonere',
'depenses_totales',
'loyer_impute',
'depenses_ht_tva_taux_plein',
'tva_taux_plein',
 'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'niveau_de_vie',
 'rev_disponible',
 'decile_rev_disponible',
 'rev_disp_yc_loyerimpute',
 'depenses_tot',
 'pondmen']

In [ ]:
year = 2024
data_year = 2017
inflators_by_year = get_inflators_by_year(rebuild = False, year_range = range(2017, 2025), data_year = data_year)
tax_benefit_system = FranceIndirectTaxationTaxBenefitSystem()
inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])

In [ ]:
survey_scenario = SurveyScenario.create(
    inflation_kwargs =  inflation_kwargs,
    baseline_tax_benefit_system = tax_benefit_system,
    reform = augmente_tous_les_taux,
    year = year,
    data_year = data_year
    )

reforme_all = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', difference = True, 
                                 variables = ['depenses_tva_taux_{}'.format(taux) for taux in ['plein','intermediaire', 'reduit', 'super_reduit']],
                                 aggfunc = 'mean')
rev_disponible_2024 = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', use_baseline= True, variables = ['niveau_de_vie'], aggfunc = 'mean')

In [ ]:
reforme_all = reforme_all.merge(rev_disponible_2024, how = 'left', right_index = True, left_index = True)

In [ ]:
recolte_taux_plein = survey_scenario.compute_aggregate(variable='tva_taux_plein',difference= True, period = year) * 1e-9
recolte_taux_inter = survey_scenario.compute_aggregate(variable='tva_taux_intermediaire',difference= True, period = year) * 1e-9
recolte_taux_reduit = survey_scenario.compute_aggregate(variable='tva_taux_reduit',difference= True, period = year) * 1e-9
recolte_taux_super_reduit = survey_scenario.compute_aggregate(variable='tva_taux_super_reduit',difference= True, period = year) * 1e-9

In [ ]:
recolte_taux_plein

In [ ]:
recolte_taux_inter

In [ ]:
recolte_taux_reduit

In [ ]:
recolte_taux_super_reduit

In [ ]:
recolte_taux_plein + recolte_taux_inter + recolte_taux_reduit + recolte_taux_super_reduit

In [ ]:
tva_total = survey_scenario.compute_aggregate(variable='tva_total', use_baseline= True, period = year) * 1e-9
depenses_ht_totales = (survey_scenario.compute_aggregate(variable = 'depenses_ht_tva_taux_plein', use_baseline= True, period = year) +
 survey_scenario.compute_aggregate(variable = 'depenses_ht_tva_taux_intermediaire', use_baseline= True, period = year) + 
 survey_scenario.compute_aggregate(variable = 'depenses_ht_tva_taux_reduit', use_baseline= True, period = year) + 
 survey_scenario.compute_aggregate(variable = 'depenses_ht_tva_taux_super_reduit', use_baseline= True, period = year)
 )* 1e-9

In [ ]:
depenses_ht_totales

In [ ]:
taux_moyen = tva_total / depenses_ht_totales
taux_moyen

In [ ]:
'depenses_ht_tva_taux_plein',
'depenses_ht_tva_taux_intermediaire',
'depenses_ht_tva_taux_reduit',
'depenses_ht_tva_taux_super_reduit',

In [ ]:
reforme_all

In [ ]:
liste_taux = ['plein','intermediaire', 'reduit', 'super_reduit']
for taux in liste_taux:
   reforme_all['Taux_effort_tva_taux_{}'.format(taux)] = reforme_all['depenses_tva_taux_{}'.format(taux)] / reforme_all['niveau_de_vie'] * 100

In [ ]:
reforme_all

In [ ]:
stacked_bar_plot(reforme_all, 
                 variables = ['depenses_tva_taux_{}'.format(taux) for taux in liste_taux],
                 labels = ['Taux plein','Taux intermédiaire','Taux réduit','Taux super réduit'],
                 title = 'Effets de l\' augmentation d\' un point de TVA',
                 xlabel = 'Décile de niveau de vie' ,
                 ylabel ='Dépenses suplémentaires (en €)',
                 colors = list(sns.color_palette("Paired")),
                 savefig = False,
                 outfile = 'Effet_net_augmentation_TVA.pdf')

In [ ]:
stacked_bar_plot(reforme_all, 
                 variables = ['Taux_effort_tva_taux_{}'.format(taux) for taux in liste_taux],
                 labels = ['Taux plein','Taux intermédiaire','Taux réduit','Taux super réduit'],
                 title = 'Effet de l\'augmentation d\'un point de TVA',
                 xlabel = 'Décile de niveau de vie' ,
                 ylabel ='Taux d\'effort (en %)',
                 colors = list(sns.color_palette("Paired")),
                 savefig = False,
                 outfile = 'Taux_effort.pdf')

In [ ]:
from openfisca_france_indirect_taxation.variables.base import get_legislation_data_frames

In [ ]:
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
categories_fiscales_data_frame.loc[categories_fiscales_data_frame['categorie_fiscale'] == 'tva_taux_super_reduit']

In [ ]:
set(categories_fiscales_data_frame['categorie_fiscale'])